<a href="https://colab.research.google.com/github/BhammarArjun/PyTorch/blob/main/PyTorch6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
 import torch
 import torch.nn as nn
 import torch.nn.functional as F


 from torch.utils.data import DataLoader
 from torchvision import datasets, transforms
 from torchvision.utils import make_grid


 import numpy as np
 import pandas as pd
 from sklearn.metrics import classification_report
 import matplotlib.pyplot as plt
 %matplotlib inline

In [2]:
 ## Convert MNIST Image into a tensor of 4 dim

 transform = transforms.ToTensor()

In [3]:
train_data = datasets.MNIST(root = 'cnn_data', train= True, download = True, transform = transform)

100%|██████████| 9912422/9912422 [00:00<00:00, 141020660.89it/s]


Extracting cnn_data/MNIST/raw/train-images-idx3-ubyte.gz to cnn_data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 107389799.49it/s]


Extracting cnn_data/MNIST/raw/train-labels-idx1-ubyte.gz to cnn_data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 47543663.01it/s]

Extracting cnn_data/MNIST/raw/t10k-images-idx3-ubyte.gz to cnn_data/MNIST/raw


100%|██████████| 4542/4542 [00:00<00:00, 6414319.45it/s]


Extracting cnn_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to cnn_data/MNIST/raw



In [4]:
test_data = datasets.MNIST(root = 'cnn_data', train= False, download = True, transform = transform)

In [5]:
train_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: cnn_data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [6]:
test_data

Dataset MNIST
    Number of datapoints: 10000
    Root location: cnn_data
    Split: Test
    StandardTransform
Transform: ToTensor()

In [7]:
pwd

'/content'

In [8]:
ls

cnn_data/  sample_data/


In [9]:
# Create a batch


train_loader = DataLoader(train_data, batch_size = 10, shuffle = True)
test_loader = DataLoader(test_data, batch_size = 10, shuffle = False)

In [10]:
## Define the CNN Network

conv1 = nn.Conv2d(1,6, 3, 1)
conv2 = nn.Conv2d(6,16, 3, 1)

In [11]:
for i, (X_train, y_train) in enumerate(train_data):
  break

In [12]:
x = X_train.view(1,1,28,28)
print(x.shape)
x = F.relu(conv1(x))
print(x.shape)
x = F.relu(conv2(x))
print(x.shape)

torch.Size([1, 1, 28, 28])
torch.Size([1, 6, 26, 26])
torch.Size([1, 16, 24, 24])


In [13]:
x = F.max_pool2d(x,2,2) # kernel of 2 and stride of 2
x.shape

torch.Size([1, 16, 12, 12])

In [14]:
## Building the entire Model


class ConvolutionNet(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(1,6,3,1)
    self.conv2 = nn.Conv2d(6,16,3,1)
    ## Fully connected layers
    self.fc1 = nn.Linear(5*5*16, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84,10)

  def forward(self, x):
    x = F.relu(self.conv1(x))
    x = F.max_pool2d(x, 2,2)
    x = F.relu(self.conv2(x))
    x = F.max_pool2d(x,2,2)

    # Flatten
    x = x.view(-1, 16*5*5)

    #FC layer

    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return F.log_softmax(x, dim=1)


In [15]:
torch.manual_seed(41)

In [24]:
# optimizer and loss function
model = ConvolutionNet()
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)

In [25]:
import time

start_time = time.time()

## create variable
epochs = 5
training_loss = []
testing_loss = []


## Create foor loop

for i in range(epochs):
  batch_corr = 0
  trn_corr = 0
  tr_los= []
  te_los = []

  for b, (x_train, y_train) in enumerate(train_loader):
    y_pred = model.forward(x_train)
    l = loss(y_pred, y_train)
    tr_los.append(l.item())

    pred = torch.argmax(y_pred, dim=1)
    batch_corr = (pred == y_train).sum()
    trn_corr += batch_corr

    optimizer.zero_grad()
    l.backward()
    optimizer.step()

  t_batch_corr =0
  tes_corr = 0


  with torch.no_grad():
    for b, (x_test, y_test) in enumerate(test_loader):
      y_val = model(x_test)
      l = loss(y_val, y_test)
      te_los.append(l.item())

      pred = torch.argmax(y_val, dim=1)
      t_batch_corr = (pred == y_test).sum()
      tes_corr += t_batch_corr

  train_loss = np.mean(tr_los)
  test_loss = np.mean(te_los)
  training_loss.append(train_loss)
  testing_loss.append(test_loss)



  print(f"Epoch {i}, Train_loss: {train_loss}, Test_loss: {test_loss}, Train_acc: {trn_corr/len(train_data)}, Test_acc: {tes_corr/ len(test_data)}")

current_time = time.time()
totl = current_time - start_time
print(f'training took : {totl/60} mins')

Epoch 0, Train_loss: 0.2468708512983078, Test_loss: 0.15479138315863628, Train_acc: 0.9307000041007996, Test_acc: 0.9632999897003174
Epoch 1, Train_loss: 0.18004380337729323, Test_loss: 0.16868354971004645, Train_acc: 0.9558500051498413, Test_acc: 0.9588000178337097
Epoch 2, Train_loss: 0.16981609616734852, Test_loss: 0.16151068081037095, Train_acc: 0.9599666595458984, Test_acc: 0.9664999842643738
Epoch 3, Train_loss: 0.17096234605140975, Test_loss: 0.11026396861554633, Train_acc: 0.9598000049591064, Test_acc: 0.9722999930381775
Epoch 4, Train_loss: 0.16615460730843168, Test_loss: 0.12818408734836453, Train_acc: 0.9607666730880737, Test_acc: 0.967199981212616
training took : 3.0352154850959776 mins


In [23]:
len(train_data)

60000